In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gzip, numpy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [2]:
batch_size = 5
nb_epochs = 100
eta = 0.01

In [3]:
((data_train, label_train), (data_test, label_test)) = torch.load(gzip.open('mnist.pkl.gz'))

C:\Users\DELL\AppData\Local\Temp\ipykernel_21808\449228702.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ((data_train, label_train), (data_test, label_test)) = torch.l

In [4]:
data_train, data_validation, label_train, label_validation = train_test_split(data_train, label_train, test_size=0.2)

In [5]:
train_dataset = torch.utils.data.TensorDataset(data_train,label_train)
test_dataset = torch.utils.data.TensorDataset(data_test,label_test)
valid_dataset = torch.utils.data.TensorDataset(data_validation,label_validation)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=1, shuffle=False)

In [15]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.Conv1 = nn.Conv2d(1,32,kernel_size=(3,3),stride=1,padding=1)
        self.relu = nn.ReLU()
        self.Conv2 = nn.Conv2d(32,64,kernel_size=(3,3),stride=1,padding=1)
        self.Maxpooling = nn.MaxPool2d(kernel_size=(2,2))
        
        self.fc1 = nn.Linear(64*7*7,128)
        self.fc2 = nn.Linear(128,10)
        
        self.dropout = nn.Dropout(0.5)
        
        
    def forward(self,x):
        
        out = self.Conv1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.Maxpooling(out)
        out = self.Conv2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.Maxpooling(out)
        
        x = torch.flatten(out, 1)
        
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.fc2(out)
        
        return out
        

In [16]:
model = CNN()

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=eta)

In [18]:
best_val_loss = float('inf')
p = 5  
t = 0

In [20]:
for epoch in range(nb_epochs):
    model.train()
    running_loss = 0
    for images ,labels in train_loader:
        optimizer.zero_grad()
        # images.shape ==> torch.Size([5, 784]) alors : 784 = 28*28 
        images = images.view(-1, 1, 28, 28)
        outputs = model(images)
        loss=criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Époch {epoch+1}/{nb_epochs}, Loss = {loss.item():.4f}")
    
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images = images.view(-1, 1, 28, 28)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    val_loss /= len(valid_loader)
    print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}")


    if val_loss < best_val_loss:
        best_val_loss = val_loss
        t = 0
    else:
        t += 1
    
    if t >= p:
        print("Early stopping!")
        break

Époch 1/100, Loss = 0.0000
Epoch 1, Validation Loss: 0.0407
Époch 2/100, Loss = 0.0000
Epoch 2, Validation Loss: 0.0336
Époch 3/100, Loss = 0.0218
Epoch 3, Validation Loss: 0.0335
Époch 4/100, Loss = 0.0000
Epoch 4, Validation Loss: 0.0347
Époch 5/100, Loss = 0.4438
Epoch 5, Validation Loss: 0.0325
Époch 6/100, Loss = 0.0028
Epoch 6, Validation Loss: 0.0343
Époch 7/100, Loss = 0.0001
Epoch 7, Validation Loss: 0.0291
Époch 8/100, Loss = 0.4802
Epoch 8, Validation Loss: 0.0308
Époch 9/100, Loss = 0.0574
Epoch 9, Validation Loss: 0.0306
Époch 10/100, Loss = 0.0020
Epoch 10, Validation Loss: 0.0279
Époch 11/100, Loss = 0.0000
Epoch 11, Validation Loss: 0.0284
Époch 12/100, Loss = 0.0001
Epoch 12, Validation Loss: 0.0297
Époch 13/100, Loss = 0.0000
Epoch 13, Validation Loss: 0.0292
Époch 14/100, Loss = 0.0064
Epoch 14, Validation Loss: 0.0264
Époch 15/100, Loss = 1.6856
Epoch 15, Validation Loss: 0.0279
Époch 16/100, Loss = 0.0000
Epoch 16, Validation Loss: 0.0261
Époch 17/100, Loss = 0.001

In [21]:
model.eval()
acc=0
all_labels = []
all_preds = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.view(-1, 1, 28, 28)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        label_multiclas = torch.argmax(labels, dim=1)
        all_labels.extend(label_multiclas.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

In [22]:
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall = recall_score(all_labels, all_preds, average='macro')
f1 = f1_score(all_labels, all_preds, average='macro')

In [23]:
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

Test Accuracy: 99.49%
Precision: 0.99
Recall: 0.99
F1-Score: 0.99
